In [1]:
from qiskit.utils import algorithm_globals
from qiskit.algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np

from docplex.mp.model import Model

from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.translators import from_docplex_mp

from qiskit.utils import algorithm_globals
from qiskit.primitives import Sampler
from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit.algorithms.optimizers import SPSA

In [2]:
# create a QUBO
qubo = QuadraticProgram()
qubo.binary_var("x")
qubo.binary_var("y")
qubo.binary_var("z")
qubo.minimize(linear=[1, -2, 3], quadratic={("x", "y"): 1, ("x", "z"): -1, ("y", "z"): 2})
print(qubo.prettyprint())

Problem name: 

Minimize
  x*y - x*z + 2*y*z + x - 2*y + 3*z

Subject to
  No constraints

  Binary variables (3)
    x y z



# Simple Maxcut with 4 nodes

## Doplex model

In [12]:


# Generate a graph of 4 nodes
n = 4
edges = [(0, 1, 1.0), (0, 2, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0)]  # (node_i, node_j, weight)

# Formulate the problem as a Docplex model
model = Model()

# Create n binary variables
x = model.binary_var_list(n)

# Define the objective function to be maximized
model.maximize(model.sum(w * x[i] * (1 - x[j]) + w * (1 - x[i]) * x[j] for i, j, w in edges))

# Fix node 0 to be 1 to break the symmetry of the max-cut solution
model.add(x[0] == 1)

# Convert the Docplex model into a `QuadraticProgram` object
problem = from_docplex_mp(model)

# Run quantum algorithm QAOA on qasm simulator
seed = 1234
algorithm_globals.random_seed = seed

spsa = SPSA(maxiter=250)
sampler = Sampler()
qaoa = QAOA(sampler=sampler, optimizer=spsa, reps=5)
algorithm = MinimumEigenOptimizer(qaoa)
result = algorithm.solve(problem)
print(result.prettyprint())  # prints solution, x=[1, 0, 1, 0], the cost, fval=4

objective function value: 4.0
variable values: x0=1.0, x1=0.0, x2=1.0, x3=0.0
status: SUCCESS


## Quadratic model

# Min-cut max-flow problem: Krauss example

## Doplex Model

In [19]:

# Generate a graph of 4 nodes, 4 edges
n = 4
edges = [(0, 1, 1.0), (0, 2, 2.0), (1, 3, 2.0), (2, 3, 1.0)]  # (node_i, node_j, weight)

# Formulate the problem as a Docplex model
model = Model()

# Create n binary variables
x = model.binary_var_list(n)

# Define the objective function to be MINIMIZED
KRAUSS_MODEL = True # add a last term for source and sink nodes
if KRAUSS_MODEL:
    model.minimize(
        model.sum(w * (x[i]*x[i] - x[i]*x[j]) for i, j, w in edges) +
        (1+1+2+2 +1)*(-x[0]*x[0] + x[0]*x[3])
    )
else:
    # fixed source and sink nodes and sink nodes
    model.minimize(
        model.sum(w * (x[i]*x[i] - x[i]*x[j]) for i, j, w in edges)
    )
    model.add(x[0] == 1)
    model.add(x[3] == 0)


print(model.export_as_lp_string())

# Convert the Docplex model into a `QuadraticProgram` object
problem = from_docplex_mp(model)

# Run quantum algorithm QAOA on qasm simulator
seed = 1234
algorithm_globals.random_seed = seed

spsa = SPSA(maxiter=250)
sampler = Sampler()
qaoa = QAOA(sampler=sampler, optimizer=spsa, reps=5)
algorithm = MinimumEigenOptimizer(qaoa)
result = algorithm.solve(problem)
print(result.prettyprint())  # prints solution, x=[1, 0, 1, 0], the cost, fval=4

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model4

Minimize
 obj: [ - 8 x1^2 - 2 x1*x2 - 4 x1*x3 + 14 x1*x4 + 4 x2^2 - 4 x2*x4 + 2 x3^2
      - 2 x3*x4 ]/2
Subject To

Bounds
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1

Binaries
 x1 x2 x3 x4
End

objective function value: -5.0
variable values: x0=1.0, x1=0.0, x2=1.0, x3=0.0
status: SUCCESS


In [21]:
# generate quantum circuit from the QAOA result
qaoa_circuit = qaoa.construct_circuit(result.x)
qaoa_circuit.draw(output='mpl')



AttributeError: 'QAOA' object has no attribute 'construct_circuit'

In [20]:
mod = from_docplex_mp(model)
print(type(mod))
print()
print(mod.prettyprint())

<class 'qiskit_optimization.problems.quadratic_program.QuadraticProgram'>

Problem name: docplex_model4

Minimize
  -4*x0^2 - x0*x1 - 2*x0*x2 + 7*x0*x3 + 2*x1^2 - 2*x1*x3 + x2^2 - x2*x3

Subject to
  No constraints

  Binary variables (4)
    x0 x1 x2 x3



In [22]:
qubitOp, offset = mod.to_ising()

In [23]:
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))

Offset: -0.25
Ising Hamiltonian:
1.0 * IIIZ
- 0.25 * IIZZ
- 0.25 * IIZI
- 0.5 * IZIZ
+ 0.25 * IZII
+ 1.75 * ZIIZ
- 1.0 * ZIII
- 0.5 * ZIZI
- 0.25 * ZZII


# Max-cut problem: LOGISMOS example

In [26]:
# read the graph from a file
import networkx as nx
import matplotlib.pyplot as plt

# G = nx.read_edgelist('./graph_samples/logismos_2d_hardconstraints.gml', nodetype=int, data=(('weight',float),))

S = 16
T = 17

# read the graph from a file from gml format
G_integer = nx.read_gml('./graph_samples/logismos_2d_hardconstraints.gml')

total_capacity = 0
for edge in G_integer.edges(data=True):
    print(edge)

    # get capacity
    print(edge[2]['capacity'])
    total_capacity += edge[2]['capacity']

print(total_capacity)

('1', '2', {'capacity': 500})
500
('1', '8', {'capacity': 500})
500
('1', '17', {'capacity': 10})
10
('2', '3', {'capacity': 500})
500
('2', '9', {'capacity': 500})
500
('2', '17', {'capacity': 10})
10
('3', '4', {'capacity': 500})
500
('3', '10', {'capacity': 500})
500
('3', '17', {'capacity': 18})
18
('4', '5', {'capacity': 500})
500
('6', '7', {'capacity': 500})
500
('6', '13', {'capacity': 500})
500
('6', '3', {'capacity': 500})
500
('6', '17', {'capacity': 5})
5
('7', '8', {'capacity': 500})
500
('7', '14', {'capacity': 500})
500
('7', '4', {'capacity': 500})
500
('7', '17', {'capacity': 1})
1
('8', '9', {'capacity': 500})
500
('8', '15', {'capacity': 500})
500
('8', '5', {'capacity': 500})
500
('8', '17', {'capacity': 17})
17
('9', '10', {'capacity': 500})
500
('11', '12', {'capacity': 500})
500
('11', '8', {'capacity': 500})
500
('11', '17', {'capacity': 10})
10
('12', '13', {'capacity': 500})
500
('12', '9', {'capacity': 500})
500
('12', '17', {'capacity': 1})
1
('13', '14', {'

In [24]:
# sum all capacities of the edges in the graph
total_capacity = 0
for edge in G_integer.edges(data=True):
    total_capacity += edge[2]['capacity']

TypeError: 'OutEdgeDataView' object is not subscriptable

In [10]:
for node in G_integer.nodes(data=True):
    print(node)

('1', {'old_label': '04'})
('2', {'old_label': '03'})
('3', {'old_label': '02'})
('4', {'old_label': '01'})
('5', {'old_label': '00'})
('6', {'old_label': '14'})
('7', {'old_label': '13'})
('8', {'old_label': '12'})
('9', {'old_label': '11'})
('10', {'old_label': '10'})
('11', {'old_label': '24'})
('12', {'old_label': '23'})
('13', {'old_label': '22'})
('14', {'old_label': '21'})
('15', {'old_label': '20'})
('16', {'old_label': 's'})
('17', {'old_label': 't'})


IndexError: list index out of range

In [35]:

# Generate a graph of 4 nodes, 4 edges
n = len(G_integer.nodes())
edges = list(G_integer.edges(data=True))  # ('node_i', 'node_j', {'capacity': cap})

# Formulate the problem as a Docplex model
model_logismos2d = Model()

# Create n binary variables
x = model_logismos2d.binary_var_list(n)

# Define the objective function QUBO to be MINIMIZED
KRAUSS_MODEL = True # add a last term for source and sink nodes
if KRAUSS_MODEL:
    model_logismos2d.minimize(
        model_logismos2d.sum(w.get('capacity') * (x[int(i)-1]*x[int(i)-1] - x[int(i)-1]*x[int(j)-1]) for i, j, w in edges) +
        (total_capacity +1)*(-x[S-1]*x[S-1] + x[S-1]*x[T-1])
    )
else:
    # fixed source and sink nodes and sink nodes
    model_logismos2d.minimize(
        model_logismos2d.sum(w.get('capacity') * (x[int(i)-1]*x[int(i)-1] - x[int(i)-1]*x[int(j)-1]) for i, j, w in edges)
    )
    model_logismos2d.add(x[S-1] == 1)
    model_logismos2d.add(x[T-1] == 0)


print(model_logismos2d.export_as_lp_string())

# Convert the Docplex model into a `QuadraticProgram` object
problem = from_docplex_mp(model_logismos2d)

# Run quantum algorithm QAOA on qasm simulator
seed = 1234
algorithm_globals.random_seed = seed

spsa = SPSA(maxiter=250)
sampler = Sampler()
qaoa = QAOA(sampler=sampler, optimizer=spsa, reps=5)
algorithm = MinimumEigenOptimizer(qaoa)
result = algorithm.solve(problem)
print(result.prettyprint())  # prints solution, x=[1, 0, 1, 0], the cost, fval=4

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model5

Minimize
 obj: [ 2020 x1^2 - 1000 x1*x2 - 1000 x1*x8 - 20 x1*x17 + 2020 x2^2 - 1000 x2*x3
      - 1000 x2*x9 - 20 x2*x17 + 2036 x3^2 - 1000 x3*x4 - 1000 x3*x6
      - 1000 x3*x10 - 36 x3*x17 + 1000 x4^2 - 1000 x4*x5 - 1000 x4*x7
      - 26 x4*x16 - 1000 x5*x8 - 2 x5*x16 + 3010 x6^2 - 1000 x6*x7 - 1000 x6*x13
      - 10 x6*x17 + 3002 x7^2 - 1000 x7*x8 - 1000 x7*x14 - 2 x7*x17 + 3034 x8^2
      - 1000 x8*x9 - 1000 x8*x11 - 1000 x8*x15 - 34 x8*x17 + 1000 x9^2
      - 1000 x9*x10 - 1000 x9*x12 - 26 x9*x16 - 1000 x10*x13 - 2 x10*x16
      + 2020 x11^2 - 1000 x11*x12 - 20 x11*x17 + 2002 x12^2 - 1000 x12*x13
      - 2 x12*x17 + 2000 x13^2 - 1000 x13*x14 - 32 x13*x16 + 1006 x14^2
      - 1000 x14*x15 - 6 x14*x17 - 2 x15*x16 - 24152 x16^2 + 24242 x16*x17 ]/2
Subject To

Bounds
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1
 0 <= x5 <= 1
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1


In [42]:
result.prettyprint()

# save model_logismos2d to file
with open('/nfs/s-iibi60/users/nale/iiai-projects/quantum-graph-remote/model_logismos2d.lp', 'w') as f:
    f.write(model_logismos2d.export_as_lp_string())

In [44]:
# load model_logismos2d from file
with open('/nfs/s-iibi60/users/nale/iiai-projects/quantum-graph-remote/model_logismos2d.lp', 'r') as f:
    model_logismos2d = f.read()
    qiskit_model_logismos2d = from_docplex_mp(f.read())

QiskitOptimizationError: 'The model is not compatible: '

In [37]:
qiskit_model_logismos2d = from_docplex_mp(model_logismos2d)
print(type(qiskit_model_logismos2d))
print()
print(qiskit_model_logismos2d.prettyprint())

<class 'qiskit_optimization.problems.quadratic_program.QuadraticProgram'>

Problem name: docplex_model5

Minimize
  1010*x0^2 - 500*x0*x1 - 10*x0*x16 - 500*x0*x7 + 1010*x1^2 - 10*x1*x16
  - 500*x1*x2 - 500*x1*x8 + 1010*x10^2 - 500*x10*x11 - 10*x10*x16 + 1001*x11^2
  - 500*x11*x12 - x11*x16 + 1000*x12^2 - 500*x12*x13 - 16*x12*x15 + 503*x13^2
  - 500*x13*x14 - 3*x13*x16 - x14*x15 - 12076*x15^2 + 12121*x15*x16 - 18*x2*x16
  + 1018*x2^2 - 500*x2*x3 - 500*x2*x5 - 500*x2*x9 - 13*x3*x15 + 500*x3^2
  - 500*x3*x4 - 500*x3*x6 - x4*x15 - 500*x4*x7 - 500*x5*x12 - 5*x5*x16
  + 1505*x5^2 - 500*x5*x6 - 500*x6*x13 - x6*x16 + 1501*x6^2 - 500*x6*x7
  - 500*x7*x10 - 500*x7*x14 - 17*x7*x16 + 1517*x7^2 - 500*x7*x8 - 500*x8*x11
  - 13*x8*x15 + 500*x8^2 - 500*x8*x9 - 500*x9*x12 - x9*x15

Subject to
  No constraints

  Binary variables (17)
    x0 x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11 x12 x13 x14 x15 x16



In [40]:
qubitOp, offset = qiskit_model_logismos2d.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))


Offset: -0.25
Ising Hamiltonian:
-252.5 * IIIIIIIIIIIIIIIIZ
- 125.0 * IIIIIIIIIIIIIIIZZ
- 127.5 * IIIIIIIIIIIIIIIZI
- 125.0 * IIIIIIIIIIIIIIZZI
- 4.5 * IIIIIIIIIIIIIIZII
- 125.0 * IIIIIIIIIIIIIZZII
+ 128.25 * IIIIIIIIIIIIIZIII
- 125.0 * IIIIIIIIIIIIZZIII
+ 250.25 * IIIIIIIIIIIIZIIII
- 125.0 * IIIIIIIIIIIZIIZII
- 376.25 * IIIIIIIIIIIZIIIII
- 125.0 * IIIIIIIIIIZIIZIII
- 250.25 * IIIIIIIIIIZIIIIII
- 125.0 * IIIIIIIIIIZZIIIII
- 125.0 * IIIIIIIIIZIIIIIIZ
- 4.25 * IIIIIIIIIZIIIIIII
- 125.0 * IIIIIIIIIZIIZIIII
- 125.0 * IIIIIIIIIZZIIIIII
- 125.0 * IIIIIIIIZIIIIIIZI
+ 253.25 * IIIIIIIIZIIIIIIII
- 125.0 * IIIIIIIIZZIIIIIII
- 125.0 * IIIIIIIZIIIIIIZII
+ 375.25 * IIIIIIIZIIIIIIIII
- 125.0 * IIIIIIIZZIIIIIIII
- 125.0 * IIIIIIZIIZIIIIIII
- 252.5 * IIIIIIZIIIIIIIIII
- 125.0 * IIIIIZIIZIIIIIIII
- 125.25 * IIIIIZIIIIIIIIIII
- 125.0 * IIIIIZZIIIIIIIIII
- 125.0 * IIIIZIIIIIIZIIIII
+ 4.0 * IIIIZIIIIIIIIIIII
- 125.0 * IIIIZIIZIIIIIIIII
- 125.0 * IIIIZZIIIIIIIIIII
- 125.0 * IIIZIIIIIIZIIIIII
+ 124.25 * III

In [41]:
!pwd

/nfs/s-iibi60/users/nale/iiai-projects/quantum-graph-remote


# QuadraticProgram and Ising Hamiltonian

In [6]:

# Make a Docplex model
from docplex.mp.model import Model

mdl = Model("docplex model")
x = mdl.binary_var("x")
y = mdl.binary_var("y")
mdl.minimize(-2 * x * y)

print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex model

Minimize
 obj: [ - 4 x*y ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1

Binaries
 x y
End



In [7]:
# load from a Docplex model to QuadraticProgram
mod = from_docplex_mp(mdl)
print(type(mod))
print()
print(mod.prettyprint())

<class 'qiskit_optimization.problems.quadratic_program.QuadraticProgram'>

Problem name: docplex model

Minimize
  -2*x*y

Subject to
  No constraints

  Binary variables (2)
    x y



In [8]:

op, offset = mod.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)

offset: -0.5
operator:
-0.5 * ZZ
+ 0.5 * IZ
+ 0.5 * ZI


In [4]:
# pauli z
pauli_z = np.array([[1, 0], [0, -1]])
# pauli x
pauli_x = np.array([[0, 1], [1, 0]])
# identity
id2 = np.eye(2)

In [16]:
H2 = -0.5*np.kron(pauli_z, pauli_z) + 0.5*np.kron(id2, pauli_z) + 0.5*np.kron(pauli_z, id2) + np.diag(offset*np.ones(4))
H2

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -2.]])

In [43]:
state_00 = np.array([[1], [0], [0], [0]])
state_01 = np.array([[0], [1], [0], [0]])
state_10 = np.array([[0], [0], [1], [0]])
state_11 = np.array([[0], [0], [0], [1]])

input = state_11
result = (input.T * H2 * input).sum()
print('Min is', result)

print('Max is', (state_00.T * H2 * input).sum())
print('Max is', (state_01.T * H2 * input).sum())
print('Max is', (state_10.T * H2 * input).sum())


Min is -2.0
Max is 0.0
Max is 0.0
Max is 0.0


In [44]:
result = np.dot(np.transpose(input), np.dot(H2, input))
print('Min is', result)

Min is [[-2.]]
